#### Infinite Generators

If you have a generator you can use `Sequence` as described above. In that case, the generator will be consumed and stored on the model as a list and its values will be validated with the sub-type of `Sequence` (e.g. `int` in `Sequence[int]`).

But if you have a generator that you don't want to be consumed, e.g. an infinite generator or a remote data loader, you can define its type with `Iterable`.

In [6]:
import itertools
from typing import Iterable
from pydantic import BaseModel, validator, ValidationError
from pydantic.fields import ModelField

In [2]:
class Model(BaseModel):
    infinite: Iterable[int]

In [3]:
def infinite_ints():
    i = 0
    while True:
        yield i
        i += 1

In [4]:
m = Model(infinite=infinite_ints())
print(m)

infinite=<generator object infinite_ints at 0x000001A2ECA10350>


In [5]:
for i in m.infinite:
    print(i)
    if i == 10:
        break

0
1
2
3
4
5
6
7
8
9
10


> ##### Warning
>
> `Iterable` fields only perform a simple check that the argument is iterable and won't be consumed. No validation of their values is performed as it cannot be done without consuming the iterable.

> Tip
>
> If you want to validate the values of an infinite generator you can create a separate model and use it while consuming the generator, reporting the validation errors as appropriate. pydantic can't validate the values automatically for you because it would require consuming the infinite generator.

##### Validating the first value

You can create a `validator` to validate the first value in an infinite generator and still not consume it entirely.

In [7]:
class ValidateModel(BaseModel):
    infinite: Iterable[int]
    
    @validator("infinite")
    def infinite_first_int(cls, iterable, field: ModelField):
        first_value = next(iterable)
        if field.sub_fields:
            sub_field = field.sub_fields[0]
            v, error = sub_field.validate(first_value, {}, loc="first_value")
            if error:
                raise ValidationError([error], cls)
        
        return itertools.chain([first_value], iterable)

In [8]:
m = ValidateModel(infinite=infinite_ints())
print(m)

infinite=<itertools.chain object at 0x000001A2ECBAF7F0>


In [9]:
def infinite_strs():
    while True:
        yield from "infinitestrs"

In [10]:
try:
    m = ValidateModel(infinite=infinite_strs())
    print(m)
except ValidationError as e:
    print(e)

1 validation error for ValidateModel
infinite -> first_value
  value is not a valid integer (type=type_error.integer)
